# Problem Set 1: Uninformed Search

**Release Date:** 25 August 2023

**Due Date:** 23:59, 2 September 2023

## Overview

In class, we discussed a number of different uninformed search algorithms. In this problem set, we get some hands-on practice by implementing them to solve some simple puzzles. In particular, we investigate the Missionaries and Cannibals problem and the Pitcher Filling problem.

Required Files:

* ps1.py

**Honour Code**: Note that plagiarism will not be condoned! You may discuss with your classmates and check the internet for references, but you MUST NOT submit code/report that is copied directly from other sources!

**IMPORTANT**: While it is possible to write and run Python code directly in Jupyter notebook, we recommend that you do this Problem Set with an IDE using the `.py` file provided. An IDE will make debugging significantly easier.

## Missionaries and Cannibals

The Missionaries and Cannibals (MnC) problem is a classic river-crossing logic puzzle. It is not difficult to solve this problem by hand when we have 3 Missionaries and 3 Cannibals. However, the solution becomes less obvious when we generalize the problem to *m* Missionaries and *c* Cannibals where *m* and *c* are some positive integers.

The Missionaries and Cannibals problem is usually stated as follows: *m* missionaries and *c* cannibals are on the right side of a river, along with a boat that can hold at most 2 people. Your goal is to find a way to get everyone to the other side, without ever leaving a group of missionaries in one place outnumbered by the cannibals in that location (or the missionaries will get eaten!). You can try it here: https://javalab.org/en/boat_puzzle_en/ to test your understanding of the problem. 

Some important points to note when solving this question:

* If the number of missionaries is zero, it does not matter how many cannibals there are even though technically they outnumber the number of missionaries. 
* You need to have at least 1 person in the boat to row the boat from one side of the river to the other. 
* At all times, both sides of the river AND the boat must have either 0 missionaries or at least as many missionaries as cannibals.

### Task 1.1  - State Representation

Propose a state representation for this problem if we want to formulate it as a search problem and define the corresponding actions.

We can define each state in a 2D array to contain the number of missionaries and cannibals on each side. Lets define the main array as A[2] and each side as another array S1[2] and S2[2] in which A[0] = S1 and A[1] = S2. S1 represents the starting side and S2 represents the other side that the people need to get across to. We can maintain each state by updating the values of S1 and S2 to keep record of the number of people on the 2 sides of the islands at all times - arbitrarily say the 0th index of the "side" arrays represent the number of missionaries and the 1st index represents the number of cannibals. 

We can check if a state is valid by checking that for both S1 and S2, the value in the 0th index is greater or equal than the value in the 1st index. 

To transition between states we can simple reduce or add the corresponding values accordingly, according to s, which I will define as the size of the boat, the number of people the boat can carry at one time - in this problem is 2 but can be generalised to be n number of people per boat.

### Task 1.2  - Initial & Goal States

What are the initial and goal states for the problem under your proposed representation?

The initial goal state would be the main array A = [[m,c], [0,0]], where m and c are equal to 3 in this problem.

The goal state would simply be A = [[0,0],[m,c]], where again m and c are both equal to 3

### Task 1.3  - Representation Invariant

What is the invariant that your representation needs to satisfy? Explain how you will use this invariant when you implement search? 

That in all states, m >= c on both sides. That means in both index 0 and 1 of the main array, the 0th index value has to be greater or equal than to the 1st index. If this invariant is not satisfied, we will rule out that state as a valid state.

So e.g [[2,2],[1,1]] is a valid state but [[1,3][2,0]] is an invalid state.


### Task 1.4  - Which Search Algorithm Should We Pick?

If we want to implement Tree Search to solve this problem, which **Tree Search** should we implement: BFS, DFS or Depth-Limited Search? Explain. You may make minor modifications to these algorithms if necessary to suit the problem.

I think since for certain values of m and c, there might not exist solutions to the problem - so ensuring that we pick a algorithm that is complete is important to ensure we do get stuck in fininite loops. For bigger values of m and c, for the same 2-sized boat, the amount of state transitions will increase exponentially as well, so we want an algorithm that isn't too consuming in space. Lastly, we are only required to give a solution and not the optimum, so it is ok to trade optimality for memory allocation.

I think that either DFS or Depth-Limited Search will work, but for bigger values of m and c, the minimum depth is unknown so we can stick with the basic structure of DFS.

Although to prevent infinite looping we need to ensure that states that have been visited are not visited again, either with some form of boolean tracker as I am assuming that this structure is a graph since we can easily visit the previous state by sending the boat back - reversing the action is a valid action in the puzzle.

### Task 1.5  - Completeness and Optimality

Derive a loose upper bound on the maximum number of valid states a game of $m$ missionaries and $c$ cannibals. Express your answer in terms of $m$ and $c$, for example, $(m+c)^{m+c}$.

Using the above upper bound, how can you ensure that Tree Search **will always terminate**? In your proposed formulation, is your Tree Search (i) complete and/or (ii) optimal? Explain.

We can start by defining the total number of valid states of a game, by observation for 3 missionaries and 3 cannibals, there are 4 possible states (0,1,2,3) and similarly for cannibals. So on one side alone we have 4x4 = 16 possible states, for an array that houses both sides we have 32 possible states.

However limiting only valid states, we notice that m = c, m = 0 or m = 3, these are the conditions for which the state is not broken - considering that both sides of the puzzle are taken into account, e.g if one state exists, its complement state must also be a valid state.

As such for m = c, m = 0 and m = 3, we have the following formula for the number of possible states on one island - 4 + 4 + 2 = 10. Generalising it for some m and c for the same conditions, where m = 3 is replaced with the max number of missionaries - we have (m + 1) + (c + 1) + m - 1 = 2m + c + 1 possible states. Multiplying it by 2 because our array houses both sides, so [[1,1],[2,2]] is distinct from [[2,2][1,1]] we get 4m + 2c + 2

Simply by keeping track of all possible valid states, whether it has been visited or not. Since we have derived a formula that gives us a finite value of states - we will be sure that our algorithm will not run into infinite loops and will hence terminate provided that it has exhausted all solutions.

In my proposed formulation, my tree search is (i) complete because we have placed a restriction on the existence of loops or infinite depth. And (ii) no it is not optimal because we do not return the solution that takes the least steps due to how DFS works.

### Task 1.6  - Implement Tree Search

Implement the function `mnc_tree_search` with **Tree** Search using your proposed representation described in Tasks 1.1, 1.2 and 1.3. 

`mnc_tree_search` takes two integers $m$ and $c$, which are the numbers of missionaries and cannibals on one side of the river initially, and returns the solution to the problem as a tuple of steps. Each step is a tuple of two numbers $m$ and $c$, indicating the number of missionaries and cannibals on the boat respectively as the boat moves from one side of the river to another. The odd steps are for the boat moving from right to left and the even steps are for the boat moving from left to right, assuming that we are counting from step zero in the tuple. Note that $1 <= c + m <= 2$, i.e. you need to have at least one person in the boat to row the boat from one side of the river to the other.


#### Worked Example

Note that there are a **range** of possible solutions. This example is just one of them. Let us consider the case of 2 missionaries and 1 cannibal. One possible solution is as follows:
- **Start:** 2 missionaries, 1 cannibal and the boat are on the LHS of the river
- **Step 1:** 2 missionaries row the boat to the RHS of the river
  
   1 cannibal is on the LHS of the river. 2 missionaries and boat are on the RHS of the river
   
   **Note:** 1 cannibal and 0 missionaries on the LHS of the river is a valid state (0 missionaries is a valid state)
- **Step 2:** 1 missionary rows the boat to the LHS of the river
   
   **Note:** 1 cannibal and 1 missionary on the LHS of the river is a valid state (no. of missionaries >= no. of cannibals)
- **Step 3:** 1 cannibal and 1 missionary row the boat to the RHS of the river
- **End:** 2 missionaries and 1 cannibal are on the RHS of the river

Hence, the solution is `((2, 0), (1, 0), (1, 1))`.

While there may be a range of solutions, for Tasks 1.6 and 1.7, we want you to return an **optimal** solution (ie. a solution of minimum moves). If there are several optimal solutions, you may return any of them. 

Hint: You may find data structures helpful for your implementation.

In [ ]:
def mnc_tree_search(m, c):  
    '''
    Solution should be the action taken from the root node (initial state) to 
    the leaf node (goal state) in the search tree.

    Parameters
    ----------    
    m: no. of missionaries
    c: no. of cannibals
    
    Returns
    ----------    
    Returns the solution to the problem as a tuple of steps. Each step is a tuple of two numbers x and y, indicating the number of missionaries and cannibals on the boat respectively as the boat moves from one side of the river to another. If there is no solution, return False.
    '''
    # TODO: add your solution here and remove `raise NotImplementedError`
    raise NotImplementedError


# Test cases 
print(mnc_tree_search(2,1))     # ((2, 0), (1, 0), (1, 1))

print(mnc_tree_search(2,2))     # ((1, 1), (1, 0), (2, 0), (1, 0), (1, 1))

print(mnc_tree_search(3,3))     # ((1, 1), (1, 0), (0, 2), (0, 1), (2, 0), (1, 1), (2, 0), (0, 1), (0, 2), (1, 0), (1, 1))

### Task 1.7 - Implement Graph Search

Next, implement the function `mnc_graph_search` with **Graph** Search using your proposed representation described in 1.1, 1.2 and 1.3. `mnc_graph_search` takes two integers $m$ and $c$, which are the numbers of missionaries and cannibals on one side of the river initially, and returns the to the problem as a tuple of steps following the format in Task 1.5.

In [ ]:
def mnc_graph_search(m,c):
    '''
    Graph search requires to deal with the redundant path: cycle or loopy path.
    Modify the above implemented tree search algorithm to accelerate your AI.

    Parameters
    ----------    
    m: no. of missionaries
    c: no. of cannibals
    
    Returns
    ----------    
    Returns the solution to the problem as a tuple of steps. Each step is a tuple of two numbers x and y, indicating the number of missionaries and cannibals on the boat respectively as the boat moves from one side of the river to another. If there is no solution, return False.
    '''
    # TODO: add your solution here and remove `raise NotImplementedError`
    raise NotImplementedError


# Test cases 
print(mnc_graph_search(2,1))     # ((2, 0), (1, 0), (1, 1))

print(mnc_graph_search(2,2))     # ((1, 1), (1, 0), (2, 0), (1, 0), (1, 1))

print(mnc_graph_search(3,3))     # ((1, 1), (1, 0), (0, 2), (0, 1), (2, 0), (1, 1), (2, 0), (0, 1), (0, 2), (1, 0), (1, 1))

print(mnc_graph_search(4, 4))    # False


### Task 1.8 - Tree vs Graph Search

Evaluate the difference in performance between Tree Search and Graph Search by timing how long each piece of code takes to run for some configurations and report your results. 

Is Graph Search (i) complete and/or (ii) optimal in your proposed formulation? Explain.

0.07839798927307129 for Tree_(3,3)
infinite loop for Tree(4,4)

0.0002989768981933594 for Graph(3,3)
0.00025582313537597656 for Graph(4,4)

Looking at the difference in timing, it is quite apparent that memoization improves the performance of the algorithm, and we are able to avoid infinite loops such as in the case of Graph(4,4), therefore:
(i) yes Graph Search is complete because it is able to throw back an error is no solution is found but all possible states have been explored
(ii) yes because Graph Search operates on BFS

## Pitcher Filling Problem

You currently have 3 pitchers $P_1$, $P_2$ and $P_3$ with capacities $c_1$, $c_2$ and $c_3$ respectively, such that $0 < c_1 < c_2 < c_3$ where each capacity is a positive integer. You also have access to a well (unlimited supply of water). Your job is to determine a sequence of steps to measure out an amount of water $a$ in any of the pitchers, where $0<a \leq c_3, a \in \mathbb{Z}_{+}$. 

<p align="center">
<img src="imgs/pitchers.jpg">
</p>

In each step, you can do one of three things:
* You can fill a pitcher $P_i$ to the brim; or
* You can empty a pitcher $P_i$; or
* You can pour water from pitcher $P_i$ to pitcher $P_j$.

In the 3rd case, we will try to pour enough water from $P_i$ to $P_j$ to fill $P_j$
**but not more**. For example, if $P_j$ is already full, then nothing happens. If the total amount of water in $P_i$ and $P_j$ is less than or equal to $c_j$, then all the water gets poured into $P_j$ and $P_i$ is emptied.

### Task 2.1  - State Representation for Pitcher Problem

Propose a state representation for this problem if we want to formulate it as a search problem and define the corresponding actions.

### Task 2.2  - Initial & Goal States

What are the initial and goal states for the problem under your proposed representation in Task 2.1?

### Task 2.3  - Implement Pitcher Search

Implement the function `pitcher_search` that can will solve the problem  **efficiently** (i.e. be mindful of your time and space complexity) using your representation in Tasks 2.1 and 2.2. `pitcher_search` takes in 4 integers integers $p_1$, $p_2$, $p_3$ and $a$, where $p_i$ are the capacities of the $i$-th pitcher and $a$ is the target amount of water, and return the answer as a tuple of steps. Each step is one of three strings:
1. "Fill $P_i$"
2. "$P_i$ => $P_j$"
3. "Empty $P_i$"
where $i$ is the label of the pitcher. If there is no solution, return `False`.


#### Worked Example

Note that there are a **range** of possible solutions. This example is just one of them. Let us consider the case where $P_1 = 2$, $P_2 = 3$, $P_3 = 7$. Assume we want to measure $a = 6$. One possible solution is as follows:
- **Start:** All 3 pitchers are empty
- **Step 1:** Fill $P_2$
- Now $P_2$ has 3 litres and $P_1$, $P_3$ have 0 litres
- **Step 2:** Pour $P_2$ into $P_3$
- Now $P_3$ has 3 litres and $P_1$, $P_2$ have 0 litres
- **Step 3:** Fill $P_2$
- Now $P_2$ has 3 litres, $P_3$ has 3 litres, and $P_1$ has 0 litres
- **Step 4:** Pour $P_2$ into $P_3$
- Now $P_3$ has 6 litres and $P_1$, $P_2$ have 0 litres

Hence, the solution is `('Fill P2', 'P2=>P3', 'Fill P2', 'P2=>P3')`.

In [ ]:
def pitcher_search(p1, p2, p3, a):
    '''
    Solution should be the action taken from the root node (initial state) to 
    the leaf node (goal state) in the search tree.

    Parameters
    ----------    
    p1: capacity of pitcher 1
    p2: capacity of pitcher 2
    p3: capacity of pitcher 3
    a: amount of water we want to measure
    
    Returns
    ----------    
    Returns the solution to the problem as a tuple of steps. Each step is a string: "Fill Pi", "Empty Pi", "Pi=>Pj". 
    If there is no solution, return False.
    '''
    # TODO: add your solution here and remove `raise NotImplementedError`
    raise NotImplementedError


# Test cases
print(pitcher_search(2,3,4,1))  # ('Fill P2', 'P2=>P1')

print(pitcher_search(1,4,9,7))  # ('Fill P3', 'P3=>P1', 'Empty P1', 'P3=>P1')

print(pitcher_search(2,3,7,8))  # False